# AI in Industry project
# Flare prediction
- Ildebrando Simeoni - ildebrando.simeoni@studio.unibo.it
- Davide Femia - davide.femia@studio.unibo.it
- Riccardo Falco - riccardo.falco2@tudio.unibo.it
- Vincenzo Collura - vincenzo.collura2@studio.unibo.it


## Libraries

In [1]:
%load_ext autoreload
%autoreload 2

from Libs.load_data import DataLoader, ClassificationDataLoader, get_dataset_split
import Libs.flares_plot as fplt
from Libs.threshold import get_labels_physic, get_labels_KDE, get_labels_quantile, get_labels_quantile_on_run
from Libs.keras_f1score import f1_m

import numpy as np
import pandas as pd
import os
from itertools import product
from tqdm import tqdm
import matplotlib.pyplot as plt
# NON SAPPIAMO SE FUNZIONANO NEL CASO INSTALLA
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix

from tensorflow import keras
# from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.initializers import Constant

2023-03-15 11:34:35.080009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 11:34:35.254190: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-15 11:34:35.254241: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-15 11:34:36.734358: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Dataframe building

In [2]:
# TO DO: sostituire tutte le seguenti celle con la funzione di Davide per creare il dataframe con cui traineremo tutti i modelli
data_loader = ClassificationDataLoader(run=1000, N=1000, s=0.5, t=0.01, d=0.2, m=1)
params = data_loader.get_params()
params

{'run': 1000,
 'sigma': [0.5],
 'theta': [0.01],
 'mu': [1],
 'delta': [0.2],
 'N': 1000}

In [3]:
Xs, best_labels = data_loader.load_data()

Loading Data
Loading Labels
Labels Loaded


In [4]:
bindexes = data_loader.get_standard_indexes()
df_train, df_val, df_test = get_dataset_split(Xs, best_labels, bindexes, window_size=100, overlap_size=0, label_treshold=1,
                                      split_on_run=True, shuffle_run=False, shuffle_window=False,
                                      test_size = 0.2, val_size=0.2, get_validation=True, random_state=42)
df_train

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,future_flare
0,1.000000,0.975000,0.817655,0.732000,0.707988,0.867460,0.901462,0.797849,0.987134,0.971664,...,0.822077,0.581339,0.584996,0.483774,0.409096,0.628952,0.611390,0.671805,0.726003,0
1,0.531933,0.435960,0.432361,0.614583,0.597788,0.458015,0.488854,0.456466,0.369691,0.327013,...,0.047971,0.036906,0.048021,0.045313,0.034575,0.046860,0.050922,0.044483,0.047844,0
2,0.047619,0.062404,0.055264,0.058290,0.063605,0.056496,0.069958,0.068656,0.086370,0.102211,...,0.097451,0.098361,0.120303,0.102902,0.089424,0.053104,0.060239,0.062240,0.071243,0
3,0.050108,0.065251,0.050780,0.080958,0.095826,0.059014,0.055918,0.042569,0.053499,0.053687,...,0.054316,0.049972,0.051270,0.052997,0.055689,0.058918,0.069401,0.072926,0.073865,0
4,0.061779,0.050530,0.048617,0.041376,0.032464,0.033079,0.027042,0.027896,0.021426,0.037713,...,0.138653,0.105199,0.093476,0.110627,0.144723,0.146574,0.134962,0.153007,0.122369,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,0.221462,0.147254,0.131077,0.143705,0.161798,0.224983,0.224525,0.242884,0.196367,0.223873,...,0.023894,0.024400,0.028591,0.028799,0.026831,0.021999,0.022993,0.026112,0.037321,0
5396,0.032010,0.036129,0.038647,0.031055,0.031585,0.031262,0.025490,0.019504,0.033207,0.041235,...,0.271392,0.300269,0.288944,0.304309,0.343996,0.225139,0.312487,0.179253,0.220719,1
5397,0.186569,0.200493,0.241656,0.259690,0.210648,0.214042,0.146446,0.162904,0.170839,0.171019,...,0.734232,1.049890,0.887703,0.530217,0.405928,0.447269,0.448764,0.432999,0.284422,0
5398,0.274142,0.358352,0.300896,0.240062,0.200058,0.297091,0.232296,0.235283,0.176103,0.183586,...,0.097322,0.101552,0.146819,0.212846,0.272731,0.339866,0.325003,0.263367,0.306598,0


In [6]:
df_val

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,future_flare
0,1.000000,0.975000,0.910019,0.676438,0.614820,0.579506,0.477794,0.431486,0.460110,0.390031,...,0.049888,0.048055,0.065509,0.038221,0.045773,0.065423,0.066232,0.065195,0.051173,0
1,0.057899,0.065381,0.071612,0.074996,0.089329,0.083485,0.053914,0.039144,0.040692,0.030953,...,0.437690,0.451191,0.432133,0.358773,0.374072,0.346017,0.329312,0.342237,0.313290,1
2,0.261432,0.314994,0.335895,0.503127,0.900586,0.986400,0.980522,0.627758,0.556900,0.753651,...,1.098298,1.300588,1.303656,1.149473,1.153794,1.292521,1.254695,0.721144,0.942372,1
3,1.292142,1.279439,1.241022,1.602959,1.501136,1.547422,1.321715,1.082633,1.052799,1.027049,...,0.064869,0.049882,0.050507,0.063416,0.094414,0.091401,0.075952,0.092999,0.069195,0
4,0.072306,0.064582,0.047139,0.055889,0.038526,0.042790,0.032053,0.040335,0.040471,0.036346,...,0.105382,0.098587,0.075462,0.104451,0.111964,0.117516,0.104060,0.086955,0.134245,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,0.050178,0.059373,0.070573,0.061751,0.073467,0.084725,0.095542,0.079884,0.060261,0.051440,...,0.042096,0.029991,0.036869,0.051091,0.050225,0.046252,0.043208,0.037943,0.030940,0
1796,0.025610,0.048762,0.044738,0.073016,0.072641,0.055691,0.072215,0.119277,0.106216,0.105193,...,0.073034,0.070488,0.070851,0.065779,0.069810,0.079705,0.089808,0.093569,0.065901,0
1797,0.051544,0.062285,0.045714,0.036734,0.053487,0.052140,0.047668,0.048065,0.042902,0.053016,...,0.035172,0.045439,0.035914,0.040348,0.038591,0.044373,0.055059,0.043246,0.043858,0
1798,0.046658,0.041885,0.051654,0.057801,0.052525,0.088456,0.095103,0.077858,0.071627,0.056934,...,0.030728,0.030415,0.033288,0.032904,0.037700,0.031231,0.029702,0.043416,0.066493,0


In [7]:
df_test

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,future_flare
0,1.000000,0.975000,1.136066,1.355935,0.902577,1.159450,1.588770,1.270311,1.152838,1.033070,...,1.594065,1.935965,1.857650,1.746675,1.364483,1.340700,1.477525,1.538220,1.445781,1
1,1.190511,0.995372,0.743651,0.715531,0.486729,0.587928,0.477054,0.630427,0.882465,1.046006,...,10.349094,7.139540,6.255424,3.838026,4.195385,5.620070,5.540732,4.487938,4.888735,1
2,4.443912,2.538261,2.420874,2.442260,2.414107,1.980401,1.394704,1.431673,1.347288,1.554923,...,0.075640,0.059306,0.044836,0.064486,0.073816,0.065052,0.052840,0.069500,0.072190,0
3,0.097886,0.108784,0.103992,0.084788,0.072219,0.092133,0.060435,0.068325,0.052513,0.045056,...,0.037423,0.038280,0.045276,0.060368,0.062827,0.066282,0.093729,0.078098,0.079507,0
4,0.072212,0.081210,0.071054,0.074997,0.071082,0.093991,0.104283,0.096287,0.127628,0.155636,...,0.244434,0.246767,0.310468,0.307184,0.278527,0.274819,0.320453,0.431115,0.420734,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,0.046822,0.056508,0.050810,0.047033,0.048166,0.057849,0.059350,0.069462,0.070214,0.069295,...,0.240207,0.212689,0.337337,0.296125,0.252650,0.241877,0.216817,0.263979,0.311165,0
1796,0.290904,0.273074,0.243153,0.265574,0.304029,0.330552,0.278046,0.204328,0.177124,0.118847,...,0.433658,0.408397,0.481602,0.443134,0.515456,0.640585,0.596393,0.497651,0.451846,0
1797,0.528299,0.612005,0.719523,0.726010,0.910680,0.871442,0.780425,0.518017,0.534429,0.517841,...,0.028357,0.020717,0.015206,0.022709,0.033863,0.032191,0.039301,0.028970,0.025797,1
1798,0.034187,0.033451,0.041974,0.048059,0.049261,0.050354,0.047208,0.057370,0.058937,0.058662,...,1.188446,0.914749,0.878636,1.033430,1.465295,1.367177,2.070074,1.761440,1.640113,1


In [8]:
train_counts = df_train['future_flare'].value_counts()
train_counts

0    4459
1     941
Name: future_flare, dtype: int64

In [9]:
val_counts = df_val['future_flare'].value_counts()
val_counts

0    1507
1     293
Name: future_flare, dtype: int64

In [10]:
test_counts = df_test['future_flare'].value_counts()
test_counts

0    1489
1     311
Name: future_flare, dtype: int64

In [11]:
total_counts = train_counts.add(val_counts).add(test_counts)
total_counts

0    7455
1    1545
Name: future_flare, dtype: int64

In [12]:
X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()
X = np.vstack((X_train, X_val, X_test))
y = np.hstack((y_train, y_val, y_test))
print(X_train.shape, X_val.shape, X_test.shape, X.shape)
print(y_train.shape, y_val.shape, y_test.shape, y.shape)

(5400, 100) (1800, 100) (1800, 100) (9000, 100)
(5400,) (1800,) (1800,) (9000,)


## Models

### Simple models

In [ ]:
class_weights = {0: 1, 1: 5}

In [ ]:
# TO DO: valutare se spostare queste funzioni nella libreria, e se tenere solo alcuni di questi modelli (i.e. quelli le cui predizioni probabilistiche non sono 1/0)

In [ ]:
def train_classifier(X_train, y_train, class_weights):

    model_svc = SVC(class_weight= class_weights, probability=True).fit(X_train, y_train)
    model_rf = RandomForestClassifier(class_weight= class_weights).fit(X_train, y_train)
    model_nb = GaussianNB().fit(X_train, y_train)
    model_lr = LogisticRegression(class_weight= class_weights).fit(X_train, y_train)
    model_knn = KNeighborsClassifier().fit(X_train, y_train)

    return model_svc, model_rf, model_nb, model_lr, model_knn

In [ ]:
def test_predictions(model_svc, model_rf, model_nb, model_lr, model_knn):

    y_pred_svc = model_svc.predict(X_test)
    y_pred_rf = model_rf.predict(X_test)
    y_pred_nb = model_nb.predict(X_test)
    y_pred_lr = model_lr.predict(X_test)
    y_pred_knn = model_knn.predict(X_test)

    return y_pred_svc, y_pred_rf, y_pred_nb, y_pred_lr, y_pred_knn

In [ ]:
def print_classification_reports(y_test, y_pred_svc, y_pred_rf, y_pred_nb, y_pred_lr, y_pred_knn):
    
    print("classification reports:\n")
    print("SVC\n\n")
    print(classification_report(y_test, y_pred_svc))
    report_svc = classification_report(y_test, y_pred_svc, output_dict=True)
    print("\nRANDOM FOREST\n\n")
    print(classification_report(y_test, y_pred_rf))
    report_rf = classification_report(y_test, y_pred_rf, output_dict=True)
    print("\nNAIVE BAYES\n\n")
    print(classification_report(y_test, y_pred_nb))
    report_nb = classification_report(y_test, y_pred_nb, output_dict=True)
    print("\nLOGISTIC REGRESSION\n\n")
    print(classification_report(y_test, y_pred_lr))
    report_lr = classification_report(y_test, y_pred_lr, output_dict=True)
    print("\nKNN\n\n")
    print(classification_report(y_test, y_pred_knn))
    report_knn = classification_report(y_test, y_pred_knn, output_dict=True)

    return report_svc, report_rf, report_nb, report_lr, report_knn

In [ ]:
def print_confusion_matrices(y_test, y_pred_svc, y_pred_rf, y_pred_nb, y_pred_lr, y_pred_knn):
    
    cm = []

    print("classification reports:\n")
    print("SVC\n\n")
    print(confusion_matrix(y_test, y_pred_svc))
    cm.append(confusion_matrix(y_test, y_pred_svc))
    print("\nRANDOM FOREST\n\n")
    print(confusion_matrix(y_test, y_pred_rf))
    cm.append(confusion_matrix(y_test, y_pred_rf))
    print("\nNAIVE BAYES\n\n")
    print(confusion_matrix(y_test, y_pred_nb))
    cm.append(confusion_matrix(y_test, y_pred_nb))
    print("\nLOGISTIC REGRESSION\n\n")
    print(confusion_matrix(y_test, y_pred_lr))
    cm.append(confusion_matrix(y_test, y_pred_lr))
    print("\nKNN\n\n")
    print(confusion_matrix(y_test, y_pred_knn))
    cm.append(confusion_matrix(y_test, y_pred_knn))

    return cm

In [ ]:
model_svc, model_rf, model_nb, model_lr, model_knn = train_classifier(X_train, y_train, class_weights)
print("Probability prediction examples:\n")
print("SVC\n\n")
print(model_svc.predict_proba(X_test))
print("\nRANDOM FOREST\n\n")
print(model_rf.predict_proba(X_test))
print("\nNAIVE BAYES\n\n")
print(model_nb.predict_proba(X_test))
print("\nLOGISTIC REGRESSION\n\n")
print(model_lr.predict_proba(X_test))
print("\nKNN\n\n")
print(model_knn.predict_proba(X_test))

In [ ]:
y_pred_svc, y_pred_rf, y_pred_nb, y_pred_lr, y_pred_knn = test_predictions(model_svc, model_rf, model_nb, model_lr, model_knn)

In [ ]:
report_svc, report_rf, report_nb, report_lr, report_knn = print_classification_reports(y_test, y_pred_svc, y_pred_rf, y_pred_nb, y_pred_lr, y_pred_knn)

classification reports:

SVC


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       835
           1       0.86      0.89      0.88        56

    accuracy                           0.98       891
   macro avg       0.93      0.94      0.93       891
weighted avg       0.98      0.98      0.98       891


RANDOM FOREST


              precision    recall  f1-score   support

           0       0.98      1.00      0.99       835
           1       0.95      0.62      0.75        56

    accuracy                           0.97       891
   macro avg       0.96      0.81      0.87       891
weighted avg       0.97      0.97      0.97       891


NAIVE BAYES


              precision    recall  f1-score   support

           0       0.99      0.98      0.99       835
           1       0.74      0.89      0.81        56

    accuracy                           0.97       891
   macro avg       0.86      0.94      0.90       891
weighted av

In [ ]:
cm = print_confusion_matrices(y_test, y_pred_svc, y_pred_rf, y_pred_nb, y_pred_lr, y_pred_knn)

classification reports:

SVC


[[827   8]
 [  6  50]]

RANDOM FOREST


[[833   2]
 [ 21  35]]

NAIVE BAYES


[[817  18]
 [  6  50]]

LOGISTIC REGRESSION


[[831   4]
 [ 13  43]]

KNN


[[831   4]
 [ 14  42]]


### CNN

### LSTM

For this test we attempted the following configuration
```
{'run': 100,
 'sigma': [0.5],
 'theta': [0.01],
 'mu': [1],
 'delta': [0.2],
 'N': 1000}
```

In [ ]:
data_loader = ClassificationDataLoader(run=100, N=1000, s=0.5, t=0.01, d=0.2, m=1)
params = data_loader.get_params()
params

{'run': 100,
 'sigma': [0.5],
 'theta': [0.01],
 'mu': [1],
 'delta': [0.2],
 'N': 1000}

In [ ]:
Xs, best_labels = data_loader.load_data(override=True)

Loading Data


100%|██████████| 100/100 [00:00<00:00, 322.71it/s]

Loading Labels
Labels Loaded


In [ ]:
bindexes = data_loader.get_standard_indexes()
df_train,df_val,df_test = get_dataset_split(Xs, best_labels, bindexes, window_size=20, overlap_size=19,
                                            label_treshold=1, split_on_run=True, shuffle_run=False, 
                                            shuffle_window=False, test_size = 0.3, val_size=0.2, 
                                            get_validation=True, random_state=42)

In [ ]:
# number of classes
print('Training set:')
print(df_train['future_flare'].value_counts(), '\n')
pos = df_train['future_flare'].value_counts()[0]
true = df_train['future_flare'].value_counts()[1]
print('validation set:')
print(df_val['future_flare'].value_counts(), '\n')
print('Test set:')
print(df_test['future_flare'].value_counts(), '\n')

Training set:
0    43768
1     4252
Name: future_flare, dtype: int64 

validation set:
0    18233
1     2347
Name: future_flare, dtype: int64 

Test set:
0    26357
1     3043
Name: future_flare, dtype: int64 



In [ ]:
X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()
X = np.vstack((X_train, X_val, X_test))
y = np.hstack((y_train, y_val, y_test))
print('X ## Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)
print('y ## Train:', y_train.shape, 'Val:', y_val.shape, 'Test:', y_test.shape)

X ## Train: (48020, 20) Val: (20580, 20) Test: (29400, 20)
y ## Train: (48020,) Val: (20580,) Test: (29400,)


In [ ]:
initial_bias = Constant([np.log(true/pos)])
# define model
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid',bias_initializer=initial_bias))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=[f1_m, 'accuracy'])

print(model.summary())

2023-03-11 16:40:53.192431: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-11 16:40:53.192637: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-11 16:40:53.192689: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (83dc2cdd3c94): /proc/driver/nvidia/version does not exist
2023-03-11 16:40:53.193367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40)               3520      
 l)                                                              
                                                                 
 dense (Dense)               (None, 30)                1230      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 20
batch_size = 32
# define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join("models", "LSTM_best_weights.h5"), save_weights_only=True, monitor="val_loss"
    ),
    EarlyStopping(monitor="val_loss", patience=3, verbose=1),
]
# fit model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

Epoch 1/20
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1501/1501 [==============================] - 37s 22ms/step - loss: 0.0517 - f1_m: 0.8051 - accuracy: 0.9811 - val_loss: 0.0219 - val_f1_m: 0.1485 - val_accuracy: 0.9926
Epoch 2/20
1501/1501 [==============================] - 48s 32ms/step - loss: 0.0277 - f1_m: 0.8642 - accuracy: 0.9883 - val_loss: 0.0182 - val_f1_m: 0.1450 - val_accuracy: 0.9934
Epoch 3/20
1501/1501 [==============================] - 44s 29ms/step - loss: 0.0254 - f1_m: 0.8643 - accuracy: 0.9893 - val_loss: 0.0196 - val_f1_m: 0.1477 - val_accuracy: 0.9926
Epoch 4/20
1501/1501 [==============================] - 41s 28ms/step - loss: 0.0240 - f1_m: 0.8847 - accuracy: 0.9900 - val_loss: 0.0211 - val_f1_m: 0.1450 - val_accuracy: 0.9917
Epoch 5/20
1501/1501 [==============================] - 37s 25ms/step - loss: 0

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.round(model.predict(X_test), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
print("F1 score: %.2f" % (f1_score(y_pred, y_test)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred)
print(result)


919/919 [==============================] - 8s 9ms/step
### Evaluation on test set ###
Accuracy: 1.00
F1 score: 0.99
[[26313    44]
 [   45  2998]]


In [ ]:
scores = model.evaluate(X_val, y_val, verbose=0)
y_pred = np.round(model.predict(X_val), 0)

print("### Evaluation on validation set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_val)))
print("F1 score: %.2f" % (f1_score(y_pred, y_val)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_val, y_pred)
print(result)


644/644 [==============================] - 5s 8ms/step
### Evaluation on validation set ###
Accuracy: 1.00
F1 score: 0.99
[[18206    27]
 [   25  2322]]


## Extra

### Model comparison

In [ ]:
# TO DO: eventuale confronto finale tra i migliori modelli ?

In [ ]:
models = ['SVC', 'RF', 'NB', 'LR', 'KNN']

fig = go.Figure([go.Bar(x=models, y=[report_svc['macro avg']['f1-score'], report_rf['macro avg']['f1-score'], report_nb['macro avg']['f1-score'], report_lr['macro avg']['f1-score'], report_knn['macro avg']['f1-score']])])
fig.update_layout(title_text='F!-macro for each model')
fig.show()

In [ ]:
rows, cols = 3, 2

fig = make_subplots(rows=rows, cols=cols)

index = 0

for i in range(rows):
    for j in range(cols):
        
        if index == 5: break

        m = [[cm[index][0][0], cm[index][0][1]], [cm[index][1][0], cm[index][1][1]]]

        fig.append_trace(go.Heatmap(z=m), row=i+1, col=j+1)

        index += 1


fig.update_layout(height=800, width=1000,title_text="Confusion matrices")
fig.show()

### Interpolation vs Extrapolation

The range of data is important when making predictions. If data points are missing from a data range, interpolation and extrapolation are used to predict the values that are in the missing range.

In [ ]:
# TODO: inserire codice Ricc e spiegare differenza tra interpolation ed extrapolation e perché con una migliorano i risultati

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.initializers import Constant
from itertools import product
from tqdm import tqdm


from Libs.load_data import DataLoader, get_dataset_split
from Libs.threshold import get_labels_physic
from Libs.keras_f1score import f1_m

2023-03-13 10:14:26.246862: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 10:14:26.482329: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 10:14:26.482381: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 10:14:27.877680: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# initialize data loader
data_loader = DataLoader(run=30, N=1000, s=0.5, t=[0.01, 0.1, 0.5, 3], d=0.2, m=1, override=True)
# get the grid
grid_X, grid_y = data_loader.get_grid()
# get params dictionary
params = data_loader.get_params()

grid_X.shape, grid_y.shape

100%|██████████| 30/30 [00:00<00:00, 48.87it/s]


((30, 1, 4, 1, 1, 1000), (30, 1, 4, 1, 1, 1000))

#### LSTM model with multiple all theta parameters

Let's start seeing what is going to happen with training and testing the NN with all the configurations of theta parameters

In [3]:
df_train,df_val,df_test = get_dataset_split(grid_X, grid_y, None, window_size=20, overlap_size=19,
                                            label_treshold=1, split_on_run=True, shuffle_run=False, 
                                            shuffle_window=False, test_size = 0.3, val_size=0.2, 
                                            get_validation=True, random_state=42)
df_train.shape, df_val.shape, df_test.shape

((54880, 21), (27440, 21), (35280, 21))

This cell shows the amounts of class 0 and 1 for each data set

In [4]:
# number of classes
print('Training set:')
print(df_train['future_flare'].value_counts(), '\n')
pos = df_train['future_flare'].value_counts()[0]
true = df_train['future_flare'].value_counts()[1]
print('validation set:')
print(df_val['future_flare'].value_counts(), '\n')
print('Test set:')
print(df_test['future_flare'].value_counts(), '\n')

Training set:
0    35397
1    19483
Name: future_flare, dtype: int64 

validation set:
0    18590
1     8850
Name: future_flare, dtype: int64 

Test set:
0    23484
1    11796
Name: future_flare, dtype: int64 



In [5]:
X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()
X = np.vstack((X_train, X_val, X_test))
y = np.hstack((y_train, y_val, y_test))
print('X ## Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)
print('y ## Train:', y_train.shape, 'Val:', y_val.shape, 'Test:', y_test.shape)

X ## Train: (54880, 20) Val: (27440, 20) Test: (35280, 20)
y ## Train: (54880,) Val: (27440,) Test: (35280,)


Let's construct now the model

In [6]:
initial_bias = Constant([np.log(true/pos)])
# define model
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid',bias_initializer=initial_bias))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=[f1_m, 'accuracy'])

print(model.summary())

2023-03-13 10:14:31.041386: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-13 10:14:31.041462: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-13 10:14:31.041492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (83dc2cdd3c94): /proc/driver/nvidia/version does not exist
2023-03-13 10:14:31.041812: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40)               3520      
 l)                                                              
                                                                 
 dense (Dense)               (None, 30)                1230      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
epochs = 20
batch_size = 32
# define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join("models", "LSTM_allTheta_checkpoint.h5"), save_weights_only=True, monitor="val_loss"
    ),
    EarlyStopping(monitor="val_loss", patience=3, verbose=1),
]
# fit model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

Epoch 1/20
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1715/1715 [==============================] - 41s 22ms/step - loss: 0.2072 - f1_m: 0.8537 - accuracy: 0.9068 - val_loss: 0.1435 - val_f1_m: 0.4336 - val_accuracy: 0.9371
Epoch 2/20
1715/1715 [==============================] - 37s 22ms/step - loss: 0.1277 - f1_m: 0.9204 - accuracy: 0.9464 - val_loss: 0.1059 - val_f1_m: 0.4451 - val_accuracy: 0.9552
Epoch 3/20
1715/1715 [==============================] - 36s 21ms/step - loss: 0.1011 - f1_m: 0.9414 - accuracy: 0.9607 - val_loss: 0.0801 - val_f1_m: 0.4790 - val_accuracy: 0.9705
Epoch 4/20
1715/1715 [==============================] - 33s 19ms/step - loss: 0.0784 - f1_m: 0.9553 - accuracy: 0.9698 - val_loss: 0.0724 - val_f1_m: 0.4754 - val_accuracy: 0.9735
Epoch 5/20
1715/1715 [==============================] - 33s 19ms/step - loss: 0

Validation set results

In [8]:
scores = model.evaluate(X_val, y_val, verbose=0)
y_pred = np.round(model.predict(X_val), 0)

print("### Evaluation on validation set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_val)))
print("F1 score: %.2f" % (f1_score(y_pred, y_val)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_val, y_pred)
print(result)

858/858 [==============================] - 6s 6ms/step
### Evaluation on validation set ###
Accuracy: 0.98
F1 score: 0.97
[[18283   307]
 [  224  8626]]


Test results

In [9]:
scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.round(model.predict(X_test), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
print("F1 score: %.2f" % (f1_score(y_pred, y_test)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred)
print(result)

1103/1103 [==============================] - 9s 8ms/step
### Evaluation on test set ###
Accuracy: 0.98
F1 score: 0.97
[[22987   497]
 [  300 11496]]


#### Interpolation

Same model as before, but now we are fitting only in the dataset coming from the generation with using just the extreme parameters: 

$\theta=0.01$ and $\theta=3$

and a fraction of the other dataset, coming from $\theta=0.1$ and $\theta=0.5$ as test set

In [10]:
params

{'run': 30,
 'sigma': [0.5],
 'theta': [0.01, 0.1, 0.5, 3],
 'mu': [1],
 'delta': [0.2],
 'N': 1000}

In [11]:
p = 'theta'
theta_train_list     = [0.01, 3]
theta_train_list_idx = [params[p].index(t) for t in theta_train_list]
theta_test_list      = [0.1, 0.5]
theta_test_list_idx  = [params[p].index(t) for t in theta_test_list]

# get the train and validation set, selecting the index for grid given the interpolation assuption
# notice that theta is the third parameter
df_train, df_val, _ = get_dataset_split(grid_X[:,:,theta_train_list_idx,:,:,:], 
                                        grid_y[:,:,theta_train_list_idx,:,:,:], 
                                        None, window_size=20, overlap_size=19,
                                        label_treshold=1, split_on_run=True, shuffle_run=False, 
                                        shuffle_window=False, test_size = 0.3, val_size=0.2, 
                                        get_validation=True, random_state=42)
# get the test set, selecting the index for grid given the interpolation assuption
# notice that theta is the third parameter
_, _, df_test = get_dataset_split(grid_X[:,:,theta_test_list_idx,:,:,:], 
                                  grid_y[:,:,theta_test_list_idx,:,:,:], 
                                  None, window_size=20, overlap_size=19,
                                  label_treshold=1, split_on_run=True, shuffle_run=False, 
                                  shuffle_window=False, test_size = 0.3, val_size=0.2, 
                                  get_validation=True, random_state=42)
df_train.shape, df_val.shape, df_test.shape

((27440, 21), (13720, 21), (17640, 21))

In [12]:
# number of classes
print('Training set:')
print(df_train['future_flare'].value_counts(), '\n')
pos = df_train['future_flare'].value_counts()[0]
true = df_train['future_flare'].value_counts()[1]
print('validation set:')
print(df_val['future_flare'].value_counts(), '\n')
print('Test set:')
print(df_test['future_flare'].value_counts(), '\n')

Training set:
0    19072
1     8368
Name: future_flare, dtype: int64 

validation set:
0    9996
1    3724
Name: future_flare, dtype: int64 

Test set:
0    10767
1     6873
Name: future_flare, dtype: int64 



In [13]:
X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()
X = np.vstack((X_train, X_val, X_test))
y = np.hstack((y_train, y_val, y_test))
print('X ## Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)
print('y ## Train:', y_train.shape, 'Val:', y_val.shape, 'Test:', y_test.shape)

X ## Train: (27440, 20) Val: (13720, 20) Test: (17640, 20)
y ## Train: (27440,) Val: (13720,) Test: (17640,)


In [14]:
initial_bias = Constant([np.log(true/pos)])
# define model
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid',bias_initializer=initial_bias))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=[f1_m, 'accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 30)                1230      
                                                                 
 dense_4 (Dense)             (None, 10)                310       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
epochs = 20
batch_size = 32
# define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join("models", "LSTM_intrpTheta_checkpoint.h5"), save_weights_only=True, monitor="val_loss"
    ),
    EarlyStopping(monitor="val_loss", patience=3, verbose=1),
]
# fit model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

Epoch 1/20
858/858 [==============================] - 22s 21ms/step - loss: 0.2894 - f1_m: 0.7462 - accuracy: 0.8680 - val_loss: 0.1762 - val_f1_m: 0.3494 - val_accuracy: 0.9170
Epoch 2/20
858/858 [==============================] - 17s 20ms/step - loss: 0.1583 - f1_m: 0.8801 - accuracy: 0.9337 - val_loss: 0.2203 - val_f1_m: 0.3943 - val_accuracy: 0.8994
Epoch 3/20
858/858 [==============================] - 20s 23ms/step - loss: 0.1177 - f1_m: 0.9153 - accuracy: 0.9530 - val_loss: 0.1013 - val_f1_m: 0.4109 - val_accuracy: 0.9601
Epoch 4/20
858/858 [==============================] - 21s 24ms/step - loss: 0.0948 - f1_m: 0.9333 - accuracy: 0.9631 - val_loss: 0.0815 - val_f1_m: 0.4184 - val_accuracy: 0.9712
Epoch 5/20
858/858 [==============================] - 18s 21ms/step - loss: 0.0875 - f1_m: 0.9438 - accuracy: 0.9680 - val_loss: 0.0853 - val_f1_m: 0.4303 - val_accuracy: 0.9734
Epoch 6/20
858/858 [==============================] - 17s 20ms/step - loss: 0.0771 - f1_m: 0.9527 - accuracy: 

Validation results

In [16]:
scores = model.evaluate(X_val, y_val, verbose=0)
y_pred = np.round(model.predict(X_val), 0)

print("### Evaluation on validation set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_val)))
print("F1 score: %.2f" % (f1_score(y_pred, y_val)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_val, y_pred)
print(result)

429/429 [==============================] - 3s 6ms/step
### Evaluation on validation set ###
Accuracy: 0.97
F1 score: 0.94
[[9983   13]
 [ 383 3341]]


Test results

In [17]:
scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.round(model.predict(X_test), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
print("F1 score: %.2f" % (f1_score(y_pred, y_test)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred)
print(result)

552/552 [==============================] - 4s 7ms/step
### Evaluation on test set ###
Accuracy: 0.98
F1 score: 0.97
[[10717    50]
 [  341  6532]]


Results are still similar to the standard case

#### Extrapolation

Same model as before, but now we are fitting only in the dataset coming from the generation without using the extreme parameters: 

$\theta=0.1$ and $\theta=0.5$

In [18]:
p = 'theta'
theta_train_list     = [0.1, 0.5]
theta_train_list_idx = [params[p].index(t) for t in theta_train_list]
theta_test_list      = [0.01, 3]
theta_test_list_idx  = [params[p].index(t) for t in theta_test_list]

# get the train and validation set, selecting the index for grid given the interpolation assuption
# notice that theta is the third parameter
df_train, df_val, _ = get_dataset_split(grid_X[:,:,theta_train_list_idx,:,:,:], 
                                        grid_y[:,:,theta_train_list_idx,:,:,:], 
                                        None, window_size=20, overlap_size=19,
                                        label_treshold=1, split_on_run=True, shuffle_run=False, 
                                        shuffle_window=False, test_size = 0.3, val_size=0.2, 
                                        get_validation=True, random_state=42)
# get the test set, selecting the index for grid given the interpolation assuption
# notice that theta is the third parameter
_, _, df_test = get_dataset_split(grid_X[:,:,theta_test_list_idx,:,:,:], 
                                  grid_y[:,:,theta_test_list_idx,:,:,:], 
                                  None, window_size=20, overlap_size=19,
                                  label_treshold=1, split_on_run=True, shuffle_run=False, 
                                  shuffle_window=False, test_size = 0.3, val_size=0.2, 
                                  get_validation=True, random_state=42)
df_train.shape, df_val.shape, df_test.shape

((27440, 21), (13720, 21), (17640, 21))

In [19]:
X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()
X = np.vstack((X_train, X_val, X_test))
y = np.hstack((y_train, y_val, y_test))
print('X ## Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)
print('y ## Train:', y_train.shape, 'Val:', y_val.shape, 'Test:', y_test.shape)

X ## Train: (27440, 20) Val: (13720, 20) Test: (17640, 20)
y ## Train: (27440,) Val: (13720,) Test: (17640,)


Same model as before

In [20]:
initial_bias = Constant([np.log(true/pos)])
# define model
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid',bias_initializer=initial_bias))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=[f1_m, 'accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 30)                1230      
                                                                 
 dense_7 (Dense)             (None, 10)                310       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
epochs = 20
batch_size = 32
# define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join("models", "LSTM_extrpTheta_checkpoint.h5"), save_weights_only=True, monitor="val_loss"
    ),
    EarlyStopping(monitor="val_loss", patience=3, verbose=1),
]
# fit model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

Epoch 1/20
858/858 [==============================] - 21s 21ms/step - loss: 0.2294 - f1_m: 0.8324 - accuracy: 0.9000 - val_loss: 0.1431 - val_f1_m: 0.5246 - val_accuracy: 0.9388
Epoch 2/20
858/858 [==============================] - 18s 21ms/step - loss: 0.1341 - f1_m: 0.9218 - accuracy: 0.9404 - val_loss: 0.1242 - val_f1_m: 0.5323 - val_accuracy: 0.9481
Epoch 3/20
858/858 [==============================] - 17s 20ms/step - loss: 0.1155 - f1_m: 0.9368 - accuracy: 0.9509 - val_loss: 0.1149 - val_f1_m: 0.5423 - val_accuracy: 0.9548
Epoch 4/20
858/858 [==============================] - 17s 20ms/step - loss: 0.1013 - f1_m: 0.9462 - accuracy: 0.9582 - val_loss: 0.0884 - val_f1_m: 0.5544 - val_accuracy: 0.9657
Epoch 5/20
858/858 [==============================] - 18s 21ms/step - loss: 0.0890 - f1_m: 0.9530 - accuracy: 0.9636 - val_loss: 0.0895 - val_f1_m: 0.5404 - val_accuracy: 0.9649
Epoch 6/20
858/858 [==============================] - 17s 20ms/step - loss: 0.0810 - f1_m: 0.9581 - accuracy: 

Validation results

In [22]:
scores = model.evaluate(X_val, y_val, verbose=0)
y_pred = np.round(model.predict(X_val), 0)

print("### Evaluation on validation set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_val)))
print("F1 score: %.2f" % (f1_score(y_pred, y_val)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_val, y_pred)
print(result)

429/429 [==============================] - 3s 6ms/step
### Evaluation on validation set ###
Accuracy: 0.98
F1 score: 0.97
[[8489  105]
 [ 170 4956]]


Test results

In [23]:
scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.round(model.predict(X_test), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
print("F1 score: %.2f" % (f1_score(y_pred, y_test)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred)
print(result)

552/552 [==============================] - 3s 6ms/step
### Evaluation on test set ###
Accuracy: 0.96
F1 score: 0.93
[[12297   420]
 [  271  4652]]
